In [1]:
import os
import json
import re
import string
from tensorflow import keras
import tensorflow as tf

In [2]:
# Load and filter data.

with open('/kaggle/input/recipe/full_format_recipes.json', 'r') as json_data:
    recipe_data = json.load(json_data)


filter_data = [
    f"Recipe for: {recipe['title']} | ingredients: {''.join(recipe['ingredients'])} | directions: {''.join(recipe['directions'])}"
    for recipe in recipe_data
              if ('title' in recipe and recipe['title'] is not None 
                  and 'directions' in recipe and recipe['directions'] is not None
                  and'ingredients' in recipe and recipe['ingredients'] is not None
                 )
              ]

In [3]:
print('Sample recipe:\n--------------\n',recipe_data[0])
print('Filtered data:\n--------------\n', filter_data[0])

Sample recipe:
--------------
 {'directions': ['1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool.', '2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper.', '3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.'], 'fat': 7.0, 'date': '2006-09-01T04:00:00.000Z', 'categories': ['Sandwich', 'Bean', 'Fruit', 'To

In [4]:
def pad_puncutation(s):
    sub = re.sub(f'([{string.punctuation}])',r'\1',s)
    sub = re.sub(' +','',sub)
    return sub


text_data = [pad_puncutation(s) for s in filter_data]

In [5]:
text_data[0]

'Recipefor:Lentil,Apple,andTurkeyWrap|ingredients:4cupslow-sodiumvegetableorchickenstock1cupdriedbrownlentils1/2cupdriedFrenchgreenlentils2stalkscelery,chopped1largecarrot,peeledandchopped1sprigfreshthyme1teaspoonkoshersalt1mediumtomato,cored,seeded,anddiced1smallFujiapple,coredanddiced1tablespoonfreshlysqueezedlemonjuice2teaspoonsextra-virginoliveoilFreshlygroundblackpeppertotaste3sheetswhole-wheatlavash,cutinhalfcrosswise,or6(12-inch)flourtortillas3/4poundturkeybreast,thinlysliced1/2headBibblettuce|directions:1.Placethestock,lentils,celery,carrot,thyme,andsaltinamediumsaucepanandbringtoaboil.Reduceheattolowandsimmeruntilthelentilsaretender,about30minutes,dependingonthelentils.(Iftheybegintodryout,addwaterasneeded.)Removeanddiscardthethyme.Drainandtransferthemixturetoabowl;letcool.2.Foldinthetomato,apple,lemonjuice,andoliveoil.Seasonwiththepepper.3.Toassembleawrap,place1lavashsheetonacleanworksurface.Spreadsomeofthelentilmixtureontheendnearestyou,leavinga1-inchborder.Topwithseveralsli

In [25]:
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(32).shuffle(10000)

In [26]:
vector_layer = keras.layers.TextVectorization(
    standardize='lower',
    max_tokens=20000,
    output_mode='int',
    output_sequence_length=200+1
)

vector_layer.adapt(text_ds)
vocab = vector_layer.get_vocabulary()



In [ ]:
def prepare_inputs_outouts(text):
    text = tf.expand_dims(text,-1)
    tockenized_text = vector_layer(text)
    x = tockenized_text[:,:-1]
    y = tockenized_text[:,1:]
    return x, y

text_ds = text_ds.map(prepare_inputs_outouts)